# make dataframe

#### I got the image information by [[siim-step1]get_imginfo](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

#### Next step, I will make the dataframe based on image information.

## my public notebook 

### [step1 get_imageinformation](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

### [step2 make_dataframe](https://www.kaggle.com/kunihikofurugori/step2-make-dataframe/edit/run/69201903).

### [step3-1 renew-imglev_ds](https://www.kaggle.com/kunihikofurugori/siim-step3-1-renew-imglev-ds)

### [step3-2 renew-studylev_ds](https://www.kaggle.com/kunihikofurugori/siim-step3-1-renew-studylev-ds)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#!pip install python-gdcm
#!pip install pylibjpeg-libjpeg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
#import pydicom
#import gdcm
#from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image
import ast

In [ ]:
ids = ["394536fc0dca_image"]

# make df_image

In [ ]:
df = pd.read_csv("../input/cov19detection-dataset/imginfo.csv")
df = df[~df.id_x.isin(ids)]
df["augflag"] = 0
index = df[(df.crop_fxmax > 0.7) & (df.crop_fymax > 0.7) & (df.crop_fxmin < 0.3) &(df.crop_fymin < 0.3)].index
df.loc[index,"augflag"] = 1
df = df.dropna(how='any')
df['boxes'] = df.boxes.apply(lambda x: ast.literal_eval(x))
df['boxnum'] = df.boxes.apply(lambda x: len(x))
df= df[df.boxnum.isin([1,2,3])]
df['box1'] = df.boxes.apply(lambda x: x[0])
df['box2'] = df.boxes.apply(lambda x: x[1] if len(x)>1 else 0)
df['box3'] = df.boxes.apply(lambda x: x[2] if len(x)>2 else 0)
df['xmin1'] = df.box1.apply(lambda x: x['x'])
df['ymin1'] = df.box1.apply(lambda x: x['y'])
df['xmax1'] = df.box1.apply(lambda x: x['x']+x['width'])
df['ymax1'] = df.box1.apply(lambda x: x['y']+x['height'])
df['area1'] = df.box1.apply(lambda x: (x['width']*x['height']))

df['xmin2'] = df.box2.apply(lambda x: x['x'] if x !=0 else 0)
df['ymin2'] = df.box2.apply(lambda x: x['y'] if x !=0 else 0)
df['xmax2'] = df.box2.apply(lambda x: x['x']+x['width'] if x !=0 else 0)
df['ymax2'] = df.box2.apply(lambda x: x['y']+x['height'] if x !=0 else 0)
df['area2'] = df.box2.apply(lambda x: x['width']*x['height'] if x !=0 else 0)

df['xmin3'] = df.box3.apply(lambda x: x['x'] if x !=0 else 0)
df['ymin3'] = df.box3.apply(lambda x: x['y'] if x !=0 else 0)
df['xmax3'] = df.box3.apply(lambda x: x['x']+x['width'] if x !=0 else 0)
df['ymax3'] = df.box3.apply(lambda x: x['y']+x['height'] if x !=0 else 0)
df['area3'] = df.box3.apply(lambda x: x['width']*x['height'] if x !=0 else 0)

df = df[df.dim2 > df.xmax1]
df = df[df.dim1 > df.ymax1]
df = df[df.dim2 > df.xmax2]
df = df[df.dim1 > df.ymax2]
df = df[df.dim2 > df.xmax3]
df = df[df.dim1 > df.ymax3]
df = df[df.xmin1>=0]
df = df[df.xmin2>=0]
df = df[df.xmin3>=0]
df = df[df.ymin1>=0]
df = df[df.ymin2>=0]
df = df[df.ymin3>=0]


df[['fracxmin1','fracxmax1','fracxmin2','fracxmax2','fracxmin3','fracxmax3']] = df.apply(lambda x: x[['xmin1','xmax1','xmin2','xmax2','xmin3','xmax3']]/x.dim2, axis=1)
df[['fracymin1','fracymax1','fracymin2','fracymax2','fracymin3','fracymax3']] = df.apply(lambda x: x[['ymin1','ymax1','ymin2','ymax2','ymin3','ymax3']]/x.dim1, axis=1)

df["fracarea1"] = df.apply(lambda x: (x.fracxmax1-x.fracxmin1)*(x.fracymax1-x.fracymin1),axis=1)
df["fracarea2"] = df.apply(lambda x: (x.fracxmax2-x.fracxmin2)*(x.fracymax2-x.fracymin2),axis=1)
df["fracarea3"] = df.apply(lambda x: (x.fracxmax3-x.fracxmin3)*(x.fracymax3-x.fracymin3),axis=1)

print(len(df))
df = df[(df.fracarea1>5e-3) | (df.fracarea1==0)]
df = df[(df.fracarea2>5e-3) | (df.fracarea2==0)]
df = df[(df.fracarea3>5e-3) | (df.fracarea3==0)]
print(len(df))

del df["Unnamed: 0"]
del df["label"]
del df["boxes"]
del df["StudyInstanceUID"]
df = df.rename(columns={'id_x': 'id'})
df.columns

In [ ]:
print(df.fracxmin1.mean(),df.fracymin1.mean(),df.fracxmax1.mean(),df.fracymax1.mean())
print(df.fracxmin2.mean(),df.fracymin2.mean(),df.fracxmax2.mean(),df.fracymax2.mean())
print(df.fracxmin3.mean(),df.fracymin3.mean(),df.fracxmax3.mean(),df.fracymax3.mean())

# safety cropping

In [ ]:
"""
cnd1 = df.fracxmax1 > df.crop_fxmax
df.loc[cnd1,"crop_fxmax"] = df.fracxmax1*1.01

cnd2 = df.fracxmax2 > df.crop_fxmax
df.loc[cnd2,"crop_fxmax"] = df.fracxmax2*1.01

cnd3 = df.fracxmax3 > df.crop_fxmax
df.loc[cnd3,"crop_fxmax"] = df.fracxmax3*1.01

cnd4 = df.fracxmin1 < df.crop_fxmin
df.loc[cnd4,"crop_fxmin"] = df.fracxmin1*0.99

cnd5 = (df.fracxmin2 < df.crop_fxmin) | (df.fracxmin2 == 0)
df.loc[cnd5,"crop_fxmin"] = df.fracxmin2*0.99

cnd6 = (df.fracxmin3 < df.crop_fxmin) | (df.fracxmin3 == 0)
df.loc[cnd6,"crop_fxmin"] = df.fracxmin3*0.99

cnd1 = df.fracymax1 > df.crop_fymax
df.loc[cnd1,"crop_fymax"] = df.fracymax1*1.01

cnd2 = df.fracymax2 > df.crop_fymax
df.loc[cnd2,"crop_fymax"] = df.fracymax2*1.01

cnd3 = df.fracymax3 > df.crop_fymax
df.loc[cnd3,"crop_fymax"] = df.fracymax3*1.01

cnd4 = df.fracymin1 < df.crop_fymin
df.loc[cnd4,"crop_fymin"] = df.fracymin1*0.99

cnd5 = (df.fracymin2 < df.crop_fymin) | (df.fracymin2 == 0)
df.loc[cnd5,"crop_fymin"] = df.fracymin2*0.99

cnd6 = (df.fracymin3 < df.crop_fymin) | (df.fracymin3 == 0)
df.loc[cnd6,"crop_fymin"] = df.fracymin3*0.99
"""

In [ ]:
area = []
area1 = df.fracarea1.values
area2 = df.fracarea2.values
area3 = df.fracarea3.values
area1 = area1[area1>0]
area2 = area2[area2>0]
area3 = area3[area3>0]
area= np.append(area,area1)
area= np.append(area,area2)
area= np.append(area,area3)
plt.hist(area,bins=np.logspace(np.log10(area.min()), np.log10(area.max()), 100),range=[area.min(),area.max()])
plt.xscale('log')
plt.title(f"areamax:{area.max():.3g},areamin:{area.min():.3g}")
plt.show()

In [ ]:
#df_image = df[df.columns[[0,5,6,7,8,9,10,11,30,31,32,33,34,35,36,37,38,39,40,41]]]
df_image= df[['id', 'path', 'fold', 'mean', 'var', 'dim1', 'dim2', 'boxnum',
       'fracxmin1', 'fracxmax1', 'fracxmin2', 'fracxmax2', 'fracxmin3',
       'fracxmax3', 'fracymin1', 'fracymax1', 'fracymin2', 'fracymax2',
       'fracymin3', 'fracymax3', 'crop_xmin', 'crop_ymin', 'crop_xmax',
       'crop_ymax', 'crop_fxmin','crop_fymin', 'crop_fxmax','crop_fymax','augflag']]
meanmax = df_image['mean'].values.max()
varmax = df_image['var'].values.max()
df_image['norm_mean'] = df_image['mean'].apply(lambda x: x/meanmax)
df_image['norm_var'] = df_image['var'].apply(lambda x: x/varmax)
df_image = df_image.reset_index(drop=True)
display(df_image)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

RANDOM_STATE = 35
fold = 1

kfold = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
df_image['test_fold'] = 0

In [ ]:
splits= kfold.split(df_image)
train_indexs = []
test_indexs = []
for i,(train_index, test_index) in enumerate(splits):
    print(train_index.shape,test_index.shape)
    train_indexs.append(train_index)
    test_indexs.append(test_index)
    df_image.loc[test_index,'test_fold'] = i

In [ ]:
df_image.to_csv("train_image.csv")
df_image.columns

# make df_study

In [ ]:
ids = ["0b858129adb4_image",
       "3d12cb6aad8b_image",
       "41e9a794b342_image",
       "681ed0b5dff2_image",
       "0eb641cb0dcd_image",
       "0f9709784c19_image",
       "3b982073ec16_image",
       "4f3d52d652dd_image",
       "9c24e37a0ef5_image",
       "1b92142f4362_image",
       "6f749e2783e1_image",
       "c3510a436bff_image",
       "394536fc0dca_image"]

In [ ]:
df = pd.read_csv("../input/cov19detection-dataset/imginfo.csv")
df = df.rename(columns={'id_x': 'id'})
df["augflag"] = 0
index = df[(df.crop_fxmax > 0.7) & (df.crop_fymax > 0.7) & (df.crop_fxmin < 0.3) &(df.crop_fymin < 0.3)].index
df.loc[index,"augflag"] = 1

df['boxes'] = df.boxes.apply(lambda x: ast.literal_eval(x))
df['boxnum'] = df.boxes.apply(lambda x: len(x))
df= df[df.boxnum.isin([1,2,3])]
df['box1'] = df.boxes.apply(lambda x: x[0])
df['box2'] = df.boxes.apply(lambda x: x[1] if len(x)>1 else 0)
df['box3'] = df.boxes.apply(lambda x: x[2] if len(x)>2 else 0)
df['xmin1'] = df.box1.apply(lambda x: x['x'])
df['ymin1'] = df.box1.apply(lambda x: x['y'])
df['xmax1'] = df.box1.apply(lambda x: x['x']+x['width'])
df['ymax1'] = df.box1.apply(lambda x: x['y']+x['height'])
df['area1'] = df.box1.apply(lambda x: (x['width']*x['height']))

df['xmin2'] = df.box2.apply(lambda x: x['x'] if x !=0 else 0)
df['ymin2'] = df.box2.apply(lambda x: x['y'] if x !=0 else 0)
df['xmax2'] = df.box2.apply(lambda x: x['x']+x['width'] if x !=0 else 0)
df['ymax2'] = df.box2.apply(lambda x: x['y']+x['height'] if x !=0 else 0)
df['area2'] = df.box2.apply(lambda x: x['width']*x['height'] if x !=0 else 0)

df['xmin3'] = df.box3.apply(lambda x: x['x'] if x !=0 else 0)
df['ymin3'] = df.box3.apply(lambda x: x['y'] if x !=0 else 0)
df['xmax3'] = df.box3.apply(lambda x: x['x']+x['width'] if x !=0 else 0)
df['ymax3'] = df.box3.apply(lambda x: x['y']+x['height'] if x !=0 else 0)
df['area3'] = df.box3.apply(lambda x: x['width']*x['height'] if x !=0 else 0)

df = df[df.dim2 > df.xmax1]
df = df[df.dim1 > df.ymax1]
df = df[df.dim2 > df.xmax2]
df = df[df.dim1 > df.ymax2]
df = df[df.dim2 > df.xmax3]
df = df[df.dim1 > df.ymax3]
df = df[df.xmin1>=0]
df = df[df.xmin2>=0]
df = df[df.xmin3>=0]
df = df[df.ymin1>=0]
df = df[df.ymin2>=0]
df = df[df.ymin3>=0]

df[['fracxmin1','fracxmax1','fracxmin2','fracxmax2','fracxmin3','fracxmax3']] = df.apply(lambda x: x[['xmin1','xmax1','xmin2','xmax2','xmin3','xmax3']]/x.dim2, axis=1)
df[['fracymin1','fracymax1','fracymin2','fracymax2','fracymin3','fracymax3']] = df.apply(lambda x: x[['ymin1','ymax1','ymin2','ymax2','ymin3','ymax3']]/x.dim1, axis=1)

df["fracarea1"] = df.apply(lambda x: (x.fracxmax1-x.fracxmin1)*(x.fracymax1-x.fracymin1),axis=1)
df["fracarea2"] = df.apply(lambda x: (x.fracxmax2-x.fracxmin2)*(x.fracymax2-x.fracymin2),axis=1)
df["fracarea3"] = df.apply(lambda x: (x.fracxmax3-x.fracxmin3)*(x.fracymax3-x.fracymin3),axis=1)

print(len(df))
df = df[(df.fracarea1>5e-3) | (df.fracarea1==0)]
df = df[(df.fracarea2>5e-3) | (df.fracarea2==0)]
df = df[(df.fracarea3>5e-3) | (df.fracarea3==0)]
print(len(df))

del df["Unnamed: 0"]
del df["label"]
del df["boxes"]


df_study = df#[df.columns[[1,4,5,6,7,8,9,11,12,13,14,-1]]]
meanmax = df_study['mean'].values.max()
varmax = df_study['var'].values.max()
df_study['norm_mean'] = df_study['mean'].apply(lambda x: x/meanmax)
df_study['norm_var'] = df_study['var'].apply(lambda x: x/varmax)
print(len(df_study),len(ids))
df_study = df_study[~df_study.id.isin(ids)]
print(len(df_study))
df_study = df_study[~df_study.duplicated(keep='last',subset=['mean', 'var'])]
print(len(df))
df_study.to_csv("train_study.csv")
df_study

# make detection or non-detection image

In [ ]:
ids = ["0b858129adb4_image",
       "3d12cb6aad8b_image",
       "41e9a794b342_image",
       "681ed0b5dff2_image",
       "0eb641cb0dcd_image",
       "0f9709784c19_image",
       "3b982073ec16_image",
       "4f3d52d652dd_image",
       "394536fc0dca_image"]

In [ ]:
df = pd.read_csv("../input/cov19detection-dataset/imginfo.csv")
df["augflag"] = 0
index = df[(df.crop_fxmax > 0.7) & (df.crop_fymax > 0.7) & (df.crop_fxmin < 0.3) &(df.crop_fymin < 0.3)].index
df.loc[index,"augflag"] = 1
df["none"] = 0
index = df[df.label=='none 1 0 0 1 1'].index
df.loc[index,"none"] = 1
index = df[(df.none==1) & (df["Negative for Pneumonia"]==0)].index
df = df.drop(index).reset_index(drop=True)
df = df.rename(columns={'id_x': 'id'})
df = df[~df.id.isin(ids)]
df = df[~df.duplicated(keep='last',subset=['mean', 'var'])].reset_index(drop=True)
df.to_csv("train_none.csv")
df